In [ ]:
from utils import cluster_scaler_client
cluster_scaler_client.scale_cluster(cluster_kind="rayclusters",replicas=4)
cluster_scaler_client.wait_until_scaling_complete(cluster_kind="rayclusters")

In [3]:
from utils import cluster_scaler_client
cluster_scaler_client.scale_cluster(cluster_kind="rayclusters",replicas=1)
cluster_scaler_client.wait_until_scaling_complete(cluster_kind="rayclusters")

http://ddl-cluster-scaler-svc.domino-field.svc.cluster.local/ddl_cluster_scaler/scale/rayclusters/ray-68b5e1f3b7910b03aaf26bb0
Status code 200
ray-68b5e1f3b7910b03aaf26bb0
http://ddl-cluster-scaler-svc.domino-field.svc.cluster.local/ddl_cluster_scaler/get/rayclusters/ray-68b5e1f3b7910b03aaf26bb0
Status code 200
Expected worker nodes 1
Current worker nodes ['ray-68b5e1f3b7910b03aaf26bb0-ray-worker-0', 'ray-68b5e1f3b7910b03aaf26bb0-ray-worker-1', 'ray-68b5e1f3b7910b03aaf26bb0-ray-worker-2', 'ray-68b5e1f3b7910b03aaf26bb0-ray-worker-3']
Scaling not yet done...
ray-68b5e1f3b7910b03aaf26bb0
http://ddl-cluster-scaler-svc.domino-field.svc.cluster.local/ddl_cluster_scaler/get/rayclusters/ray-68b5e1f3b7910b03aaf26bb0
Status code 200
Expected worker nodes 1
Current worker nodes ['ray-68b5e1f3b7910b03aaf26bb0-ray-worker-0', 'ray-68b5e1f3b7910b03aaf26bb0-ray-worker-1', 'ray-68b5e1f3b7910b03aaf26bb0-ray-worker-2', 'ray-68b5e1f3b7910b03aaf26bb0-ray-worker-3']
Scaling not yet done...
ray-68b5e1f3b7910

True